In [ ]:
import pandas as pd
import matplotlib as plt
import rasterio
import numpy as np

Crime data loading

In [ ]:
df = pd.read_csv("C:/Users/User/PycharmProjects/SafeRoute/NYPD_Arrests_Data__Historic_.csv") # TODO change to relative path
df = df.replace('(null)', np.nan)
df = df.replace('nan', np.nan)
df = df.dropna(axis=0)
df

ALL OFFENCE DESCRIPTION

In [ ]:
# np.unique(np.array())
list(set(df['OFNS_DESC']))

ASSIGNING ARBITRARY DANGER VALUES FOR CRIME

In [ ]:
danger_scores = {
    'HOMICIDE-NEGLIGENT,UNCLASSIFIED': 10,
    'SEX CRIMES': 9,
    'GRAND LARCENY': 3,
    'ROBBERY': 8,
    'UNLAWFUL POSS. WEAP. ON SCHOOL GROUNDS': 6,
    'KIDNAPPING & RELATED OFFENSES': 9,
    'BURGLARY': 6,
    'DANGEROUS DRUGS': 5,
    'FRAUDS': 2,
    'RAPE': 12,
    'FELONY SEX CRIMES': 10,
    'ASSAULT 3 & RELATED OFFENSES': 8,
    'UNAUTHORIZED USE OF A VEHICLE 3 (UUV)': 4,
    'FELONY ASSAULT': 9,
    'MURDER & NON-NEGL. MANSLAUGHTER': 15,
    'THEFT-FRAUD': 3,
    'PETIT LARCENY': 2
}

FORMING NEW DATAFRAME

In [ ]:
new_df = df.loc[df['OFNS_DESC'].isin(danger_scores.keys()), ['ARREST_DATE', 'Latitude', 'Longitude']]
new_df

In [ ]:
new_df['Danger Score'] = df.loc[df['OFNS_DESC'].isin(danger_scores.keys()), 'OFNS_DESC'].map(danger_scores)
new_df

In [ ]:
new_df.sort_values(by=['Longitude'])

In [ ]:
(new_df['Latitude'].min(), new_df['Latitude'].max(), new_df['Longitude'].min(), new_df['Longitude'].max())

OPENING CLUSTERED GEOTIF POPULATION FILE

In [ ]:
with rasterio.open("C:/Users/User/PycharmProjects/SafeRoute/clustered.tif") as src:
    data = src.read()
    image = src.read(1)
    clusters = src.read(2)
    bounds = src.bounds
    

In [ ]:
bounds

In [ ]:
(abs(bounds[2] - bounds[0]), abs(bounds[3] - bounds[1]))

In [ ]:
step = (abs(bounds[2] - bounds[0])/np.shape(clusters)[1] + abs(bounds[3] - bounds[1])/np.shape(clusters)[0]) / 2
step

COUNTING POPULATION PER CLUSTER

In [ ]:
pop_dict = {c: [] for c in np.unique(clusters)}
pop_count = {c: 0 for c in np.unique(clusters)}
for iy, ix in np.ndindex(image.shape):
    ap = image[iy, ix] if image[iy, ix] > 0 else 0
    pop_dict[clusters[iy, ix]].append(ap)
for k, v in pop_dict.items():
    pop_count[k] = sum(pop_dict[k])/len(pop_dict[k]) if len(pop_dict[k]) > 0 else 0
pop_count

In [ ]:
cluster_score = {c: 0 for c in np.unique(clusters)}
cluster_crime_num = {c: 0 for c in np.unique(clusters)}
cluster_score

CALCULATING CRIME NUMBER AND SCORE PER CLUSTER

In [ ]:
for row in new_df.itertuples():
    p = (row.Longitude, row.Latitude) # point declaration
    if abs(p[0])>abs(bounds[0]) or abs(p[0])<abs(bounds[2]): 
        continue
    elif abs(p[1])<abs(bounds[1]) or abs(p[1])>abs(bounds[3]): 
        continue
    cluster_score[clusters[round((abs(p[1]) - abs(bounds[1]))/step) - 1, round((abs(p[0]) - abs(bounds[2]))/step) - 1]] += row._4
    cluster_crime_num[clusters[round((abs(p[1]) - abs(bounds[1]))/step) - 1, round((abs(p[0]) - abs(bounds[2]))/step) - 1]] += 1

In [ ]:
cluster_score

In [ ]:
cluster_score[-1] = 0
cluster_crime_num[-1] = 0
cluster_score

FINDING AREA PER CLUSTER

In [ ]:
cluster_area = {c: (np.count_nonzero(clusters == c))*100 for c in np.unique(clusters)}
cluster_area

In [ ]:
for k, v in cluster_score.items():
    cluster_score[k] = v/cluster_area[k]
    cluster_crime_num[k] = cluster_crime_num[k]/cluster_area[k]

cluster_score

In [ ]:
fin_df = pd.DataFrame({"pop_count_adj": pop_count ,'crime_number_adj': cluster_crime_num, 'crime_score_adj': cluster_score})
fin_df.to_csv('crime_score.csv', index_label='cluster')
fin_df

MERGING DATAFRAMES FOR FINAL FILE

In [ ]:
df1 = pd.read_csv('crime_score.csv')
df2 = pd.read_csv('average_income.csv')
merged_df = pd.merge(df1, df2, on='cluster', how='outer')
merged_df.to_csv('all_data.csv', index=False)


ADJUSTING INCOME

In [ ]:
edited_df = merged_df[(merged_df != 0).all(axis=1)]
i_l = edited_df['income'].tolist()
pop_list = edited_df['pop_count_adj'].tolist()
crime_list = edited_df['crime_score_adj'].tolist()

In [ ]:
v_min = min([value for value in i_l if value >= 1000])
v_max = max([value for value in i_l if value >= 1000])
for i in range(len(i_l)):
    i_l[i] = ((i_l[i] - v_min) if v_min < i_l[i] else 0)
    i_l[i] = i_l[i]/v_max

In [ ]:
v_max = max(pop_list)
for i in range(len(pop_list)):
    pop_list[i] = pop_list[i]/v_max

In [ ]:
v_max = max(crime_list)
v_min = min(crime_list)
for i in range(len(crime_list)):
    crime_list[i] = (crime_list[i] - v_min)/v_max

In [ ]:
edited_df['income'] = i_l
edited_df['pop_count_adj'] = pop_list
edited_df['crime_score_adj'] = pop_list
edited_df.to_csv('edited_data.csv', index=False)

In [ ]:
df666 = pd.read_csv('edited_data.csv')
df666